In [ ]:
import glob
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


### PATH

#path = ""

deepCeres = path + "deepCeres/"

acapulco = path + "acapulco/"
#acapulco_source = "HNU/derivatives/acapulco/"

suit = path + "vbm_suit/"

ceres = path + "ceres/"

In [45]:
# Columns order
finalColumns = ['ID', 'ses', 'age', 'sex', 'ICV', 'Left.I.V', 'Right.I.V',
       'Left.VI', 'Right.VI',
       'Left.Crus.I', 'Right.Crus.I', 'Left.Crus.II', 'Right.Crus.II',
       'Left.VIIB', 'Right.VIIB', 'Left.VIIIA', 'Right.VIIIA', 'Left.VIIIB',
       'Right.VIIIB', 'Left.IX', 'Right.IX', 'Left.X', 'Right.X']

# Preprocessing with all good subjects for each method

# Ceres

In [ ]:
# Get subjects list and information
subjects = pd.read_csv(path+'subjects_Ceres.csv')
subjects_list = subjects['ID'].tolist()

print(subjects_list)
subjects

['sub-0025428', 'sub-0025432', 'sub-0025434', 'sub-0025435', 'sub-0025439', 'sub-0025440', 'sub-0025444', 'sub-0025445', 'sub-0025446', 'sub-0025447', 'sub-0025448', 'sub-0025449', 'sub-0025450', 'sub-0025452', 'sub-0025454', 'sub-0025455']


,ID,age,sex
0,sub-0025428,28,F
1,sub-0025432,21,M
2,sub-0025434,24,M
3,sub-0025435,30,F
4,sub-0025439,24,F
5,sub-0025440,24,M
6,sub-0025444,27,F
7,sub-0025445,23,F
8,sub-0025446,30,M
9,sub-0025447,26,M


In [ ]:
# Columns to keep and convert
col = {
    'ICV cm3':'ICV',
    'I-II right cm3':'Right.I.II',
    'I-II left cm3':'Left.I.II',
    'III right cm3':'Right.III',
    'III left cm3':'Left.III',
    'IV right cm3':'Right.IV',
    'IV left cm3':'Left.IV',
    'V right cm3':'Right.V',
    'V left cm3':'Left.V',
    'VI right cm3':'Right.VI',
    'VI left cm3':'Left.VI',
    'Crus I right cm3':'Right.Crus.I',
    'Crus I left cm3':'Left.Crus.I',
    'Crus II right cm3':'Right.Crus.II',
    'Crus II left cm3':'Left.Crus.II',
    'VIIB right cm3':'Right.VIIB',
    'VIIB left cm3':'Left.VIIB',
    'VIIIA right cm3':'Right.VIIIA',
    'VIIIA left cm3':'Left.VIIIA',
    'VIIIB right cm3':'Right.VIIIB',
    'VIIIB left cm3':'Left.VIIIB',
    'IX right cm3':'Right.IX',
    'IX left cm3':'Left.IX',
    'X right cm3':'Right.X',
    'X left cm3':'Left.X',
}

In [ ]:
# List to store DataFrames
all_data = []

# Loop through subjects and sessions
for sub in subjects_list:
    for ses in range(1,11):
        ses_str = f"{ses:02d}"
        
        # Get the matching file (for given subject and given session)
        dataFile = glob.glob(os.path.join(ceres, 'output', f"{sub}_ses-{ses_str}_T1w_job*_archive", "report_job*.csv"))[0]

        # Read CSV
        data = pd.read_csv(dataFile, sep=';')

        # Keep only the columns in col and rename them
        data = data[list(col.keys())].rename(columns=col)

        # Group I.II, III, IV and V into I.V
        data.insert(4, "Left.I.V", data["Left.I.II"] + data["Left.III"] + data["Left.IV"] + data["Left.V"])
        data.insert(4, "Right.I.V", data["Right.I.II"] + data["Right.III"] + data["Right.IV"] + data["Right.V"])
        data = data.drop(["Left.I.II", "Left.III", "Left.IV", "Left.V", "Right.I.II", "Right.III", "Right.IV", "Right.V"], axis = 1)

        # Add Subject ID and Session columns at the beginning
        data.insert(0, 'ses', ses_str)
        data.insert(0, 'ID', sub)

        # Append to the list
        all_data.append(data)

# Concatenate all DataFrames into one
ceres_df = pd.concat(all_data, ignore_index=True)

# Merge with age and sex
ceres_df= ceres_df.merge(subjects, on="ID", how="left")

# Sort columns
ceres_df = ceres_df[finalColumns]

# Save it
#ceres_df.to_csv(ceres+'all_volumes_cm3_ceres_ALLGOODDATA.csv', index=False)

ceres_df

,ID,ses,age,sex,ICV,Left.I.V,Right.I.V,Left.VI,Right.VI,Left.Crus.I,...,Left.VIIB,Right.VIIB,Left.VIIIA,Right.VIIIA,Left.VIIIB,Right.VIIIB,Left.IX,Right.IX,Left.X,Right.X
0,sub-0025428,01,28,F,1356.4357,8.544582,8.471637,8.9985,9.5202,13.4904,...,4.0988,5.8708,6.1675,4.7996,4.6191,2.8119,4.1185,4.1733,0.76217,0.76363
1,sub-0025428,02,28,F,1366.2293,8.619207,8.284889,9.3166,9.6142,13.7456,...,4.4173,3.9308,6.0729,5.7547,4.6002,4.8214,4.2762,4.5613,0.67754,0.76646
2,sub-0025428,03,28,F,1362.6981,8.630491,8.367688,9.1956,9.8435,14.0848,...,4.2494,4.0371,6.2031,5.8261,4.6081,4.8430,4.1710,4.5027,0.70274,0.74666
3,sub-0025428,04,28,F,1369.8166,8.507694,8.410795,9.1906,9.7826,13.6649,...,4.7743,4.0400,5.7368,5.9833,4.6621,5.0157,4.1485,4.4919,0.68959,0.75048
4,sub-0025428,05,28,F,1360.8444,8.616637,8.130018,9.2284,9.9946,14.3354,...,4.4075,4.1506,6.1967,6.0598,4.4616,4.8385,4.2582,4.4872,0.66957,0.72811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,sub-0025455,06,25,M,1485.0321,7.313601,5.681421,8.3756,9.4175,9.7702,...,4.0195,4.8932,6.1187,5.1283,4.5453,3.4591,3.4511,3.8996,0.57169,0.62402
156,sub-0025455,07,25,M,1482.4569,7.027703,5.701556,8.4550,9.2323,9.9022,...,3.9602,5.0549,6.4989,5.7240,4.7824,3.4666,3.5620,3.7719,0.58097,0.58899
157,sub-0025455,08,25,M,1494.1008,6.972269,5.806073,8.7405,9.2614,10.0760,...,3.9842,4.4417,6.6473,5.9169,4.6407,3.8710,3.5869,3.7193,0.55220,0.60597
158,sub-0025455,09,25,M,1490.8639,7.096977,5.708222,8.6374,9.7394,9.7828,...,3.7958,5.0930,6.5299,5.6255,4.7549,3.4505,3.5236,3.8979,0.57910,0.65138


In [ ]:
# Scale data
scaler = MinMaxScaler()

# Divide by ICV and scale
ceres_scaled = ceres_df.copy()
ceres_scaled.iloc[:,4:] = scaler.fit_transform(ceres_scaled.iloc[:,4:].div(ceres_scaled.ICV, axis=0))

# Save the dataframe
#ceres_scaled.to_csv(ceres+'all_volumes_cm3_ceres_scaled_ALLGOODDATA.csv', index=False)

ceres_scaled

,ID,ses,age,sex,ICV,Left.I.V,Right.I.V,Left.VI,Right.VI,Left.Crus.I,...,Left.VIIB,Right.VIIB,Left.VIIIA,Right.VIIIA,Left.VIIIB,Right.VIIIB,Left.IX,Right.IX,Left.X,Right.X
0,sub-0025428,01,28,F,0.0,0.796936,0.912066,0.873554,0.808657,0.851797,...,0.399213,1.000000,0.496051,0.172903,0.661024,0.010255,0.938179,0.827542,1.000000,1.000000
1,sub-0025428,02,28,F,0.0,0.800433,0.843669,0.934881,0.813882,0.880872,...,0.528705,0.291941,0.455503,0.481439,0.642258,0.648803,1.000000,0.988164,0.717020,0.991466
2,sub-0025428,03,28,F,0.0,0.809531,0.872494,0.911339,0.866581,0.950098,...,0.458385,0.333645,0.498134,0.510433,0.649384,0.659766,0.954399,0.967099,0.801855,0.934653
3,sub-0025428,04,28,F,0.0,0.764258,0.872328,0.898523,0.843411,0.859409,...,0.683115,0.327165,0.353170,0.552814,0.660105,0.706959,0.933058,0.951735,0.749182,0.934398
4,sub-0025428,05,28,F,0.0,0.808957,0.809817,0.922359,0.900735,1.000000,...,0.532130,0.376314,0.498730,0.591747,0.598821,0.660439,0.999444,0.962875,0.700316,0.878671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,sub-0025455,06,25,M,0.0,0.289225,0.000000,0.544566,0.617382,0.004351,...,0.206275,0.495867,0.326191,0.133946,0.491945,0.122662,0.468006,0.551153,0.241105,0.379730
156,sub-0025455,07,25,M,0.0,0.221144,0.007625,0.565538,0.585195,0.029643,...,0.184659,0.551888,0.431160,0.320718,0.573032,0.126655,0.520047,0.501128,0.270824,0.280236
157,sub-0025455,08,25,M,0.0,0.193792,0.022694,0.614075,0.576984,0.045822,...,0.181747,0.338645,0.457105,0.366064,0.514160,0.237368,0.518687,0.467381,0.175128,0.316160
158,sub-0025455,09,25,M,0.0,0.228434,0.001135,0.595389,0.671382,0.000000,...,0.107907,0.554975,0.429596,0.280490,0.555055,0.116136,0.494103,0.544155,0.255965,0.452323


# Acapulco

In [ ]:
# Get subjects list and information
subjects = pd.read_csv(path+'subjects_Acapulco.csv')
subjects_list = subjects['ID'].tolist()

print(subjects_list)
subjects

['sub-0025434', 'sub-0025435', 'sub-0025440', 'sub-0025442', 'sub-0025444', 'sub-0025446', 'sub-0025447', 'sub-0025449', 'sub-0025454', 'sub-0025455']


,ID,age,sex
0,sub-0025434,24,M
1,sub-0025435,30,F
2,sub-0025440,24,M
3,sub-0025442,27,F
4,sub-0025444,27,F
5,sub-0025446,30,M
6,sub-0025447,26,M
7,sub-0025449,22,M
8,sub-0025454,26,M
9,sub-0025455,25,M


In [ ]:
# Columns to keep and convert
col = {
    'eTIV':'ICV',
    'Total_Cerebel_Vol':'Total_Cereb',
    'Rigt.IV' : 'Right.IV' # Error in ACAPULCO's outputs
}

# Column to drop
col_drop = ['Total_Cereb', 'Background', 'Corpus.Medullare', 'Vermis.IX', 'Vermis.VI', 'Vermis.VII', 'Vermis.VIII', 'Vermis.X']

In [ ]:
# List to store DataFrames
all_data = []

# Loop through sessions
for ses in range(1, 11):
    ses_str = f"{ses:02d}"

    # Read CSV for given session
    data = pd.read_csv(acapulco_source + 'ses-' + ses_str + '/QC+vols/Cerebel_vols.csv', index_col=0)

    # Keep chosen subjects
    data = data[data["ID"].isin(subjects_list)].reset_index(drop=True)

    # Rename the columns in col and drop col_drop
    data = data.rename(columns=col)
    data = data.drop(col_drop, axis=1)

    # Group I.III, IV and V into I.V
    data.insert(4, "Left.I.V", data["Left.I.III"] + data["Left.IV"] + data["Left.V"])
    data.insert(4, "Right.I.V", data["Right.I.III"] + data["Right.IV"] + data["Right.V"])
    data = data.drop(["Left.I.III", "Left.IV", "Left.V", "Right.I.III", "Right.IV", "Right.V"], axis = 1)

    # Add Session column at the beginning
    data.insert(1, 'ses', ses_str)

    # Append to the list
    all_data.append(data)

# Concatenate all DataFrames into one
acapulco_df = pd.concat(all_data, ignore_index=True)

# Sort given id and session
acapulco_df = acapulco_df.sort_values(by=["ID", "ses"], ascending=[True, True]).reset_index(drop=True)

# Merge with age and sex
acapulco_df= acapulco_df.merge(subjects, on="ID", how="left")

# Sort columns
acapulco_df = acapulco_df[finalColumns]

for i in range(4,acapulco_df.shape[1]):
    acapulco_df.isetitem(i, acapulco_df.iloc[:,i].astype(float))

# Save it
#acapulco_df.to_csv(acapulco+'all_volumes_cm3_acapulco_ALLGOODDATA.csv', index=False)

acapulco_df

,ID,ses,age,sex,ICV,Left.I.V,Right.I.V,Left.VI,Right.VI,Left.Crus.I,...,Left.VIIB,Right.VIIB,Left.VIIIA,Right.VIIIA,Left.VIIIB,Right.VIIIB,Left.IX,Right.IX,Left.X,Right.X
0,sub-0025434,01,24,M,1.343446e+06,7937.0,7227.0,7128.0,7968.0,10977.0,...,5346.0,5079.0,4545.0,4665.0,3739.0,3217.0,2365.0,2350.0,354.0,358.0
1,sub-0025434,02,24,M,1.343372e+06,7859.0,7349.0,7462.0,7932.0,11118.0,...,5346.0,5003.0,5130.0,4963.0,3676.0,3064.0,2429.0,2405.0,352.0,371.0
2,sub-0025434,03,24,M,1.332420e+06,7706.0,7588.0,7558.0,7593.0,10475.0,...,5222.0,5108.0,5080.0,4812.0,4027.0,3267.0,2389.0,2432.0,365.0,402.0
3,sub-0025434,04,24,M,1.349433e+06,7855.0,7310.0,7548.0,8131.0,11026.0,...,5647.0,5012.0,4602.0,4877.0,3634.0,3433.0,2360.0,2437.0,365.0,382.0
4,sub-0025434,05,24,M,1.339030e+06,7635.0,7277.0,7378.0,7783.0,10772.0,...,4899.0,4616.0,5317.0,4942.0,3820.0,3135.0,2352.0,2332.0,356.0,370.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,sub-0025455,06,25,M,1.546649e+06,6925.0,6418.0,9392.0,9149.0,10762.0,...,5541.0,6173.0,6025.0,4413.0,3116.0,3456.0,3125.0,3162.0,407.0,416.0
96,sub-0025455,07,25,M,1.552172e+06,7219.0,6765.0,9179.0,9210.0,11306.0,...,6126.0,5779.0,5566.0,4373.0,2873.0,3343.0,3067.0,3119.0,469.0,415.0
97,sub-0025455,08,25,M,1.550504e+06,7061.0,6686.0,9065.0,8875.0,11458.0,...,6109.0,5975.0,6041.0,4687.0,2893.0,3275.0,3276.0,3078.0,377.0,428.0
98,sub-0025455,09,25,M,1.561325e+06,7253.0,6911.0,9176.0,9118.0,11103.0,...,6301.0,6220.0,5851.0,5080.0,2800.0,3254.0,3094.0,3244.0,436.0,447.0


In [ ]:
# Scale data
scaler = MinMaxScaler()

# Divide by ICV and scale
acapulco_scaled = acapulco_df.copy()
acapulco_scaled.iloc[:, 4:] = scaler.fit_transform(acapulco_scaled.iloc[:, 4:].div(acapulco_scaled.ICV, axis=0))

# Save the dataframe
#acapulco_scaled.to_csv(acapulco+'all_volumes_cm3_acapulco_scaled_ALLGOODDATA.csv', index=False)

acapulco_scaled

,ID,ses,age,sex,ICV,Left.I.V,Right.I.V,Left.VI,Right.VI,Left.Crus.I,...,Left.VIIB,Right.VIIB,Left.VIIIA,Right.VIIIA,Left.VIIIB,Right.VIIIB,Left.IX,Right.IX,Left.X,Right.X
0,sub-0025434,01,24,M,0.0,0.767530,0.598178,0.397863,0.790578,0.628716,...,0.420060,0.245213,0.404210,0.552149,0.894147,0.617016,0.275993,0.361725,0.334403,0.366802
1,sub-0025434,02,24,M,0.0,0.747330,0.635613,0.544393,0.769896,0.669789,...,0.420172,0.216194,0.625073,0.682542,0.873459,0.542686,0.315047,0.388265,0.327504,0.415832
2,sub-0025434,03,24,M,0.0,0.723981,0.727785,0.613683,0.609056,0.508251,...,0.389166,0.272522,0.621961,0.633819,1.000000,0.654481,0.302646,0.410886,0.383235,0.545018
3,sub-0025434,04,24,M,0.0,0.737101,0.613642,0.567222,0.864380,0.628737,...,0.525580,0.211016,0.418014,0.635367,0.854245,0.714660,0.266572,0.398400,0.367075,0.450795
4,sub-0025434,05,24,M,0.0,0.695445,0.620809,0.518069,0.697928,0.579559,...,0.255306,0.073624,0.702151,0.680368,0.924981,0.582164,0.272800,0.356762,0.345458,0.416585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,sub-0025455,06,25,M,0.0,0.267083,0.092880,0.849208,0.778352,0.156239,...,0.216244,0.353647,0.663992,0.188521,0.554048,0.512421,0.488777,0.552627,0.332756,0.379396
96,sub-0025455,07,25,M,0.0,0.327773,0.178666,0.755708,0.792611,0.283161,...,0.403316,0.215659,0.507049,0.167424,0.481589,0.459659,0.452326,0.530006,0.515280,0.371296
97,sub-0025455,08,25,M,0.0,0.293875,0.159659,0.716172,0.628816,0.324415,...,0.399865,0.282799,0.664315,0.288142,0.488179,0.432518,0.564340,0.514304,0.239160,0.415167
98,sub-0025455,09,25,M,0.0,0.325853,0.206588,0.734168,0.719405,0.215857,...,0.449010,0.349831,0.588931,0.423666,0.456094,0.414164,0.456997,0.574164,0.408093,0.467052


## DeepCeres

In [44]:
# Get subjects list and information
subjects = pd.read_csv(path+'subjects_DeepCERES.csv')
subjects_list = subjects['ID'].tolist()

print(subjects_list)
subjects

['sub-0025428', 'sub-0025434', 'sub-0025435', 'sub-0025439', 'sub-0025440', 'sub-0025441', 'sub-0025442', 'sub-0025443', 'sub-0025444', 'sub-0025446', 'sub-0025447', 'sub-0025448', 'sub-0025449', 'sub-0025450', 'sub-0025452', 'sub-0025454', 'sub-0025455', 'sub-0025456']


,ID,age,sex
0,sub-0025428,28,F
1,sub-0025434,24,M
2,sub-0025435,30,F
3,sub-0025439,24,F
4,sub-0025440,24,M
5,sub-0025441,24,F
6,sub-0025442,27,F
7,sub-0025443,25,M
8,sub-0025444,27,F
9,sub-0025446,30,M


In [46]:
# Columns to keep and convert
col = {
    'ICV total volume cm3':'ICV',
    'Lobules I-II right volume cm3':'Right.I.II',
    'Lobules I-II left volume cm3':'Left.I.II',
    'Lobule III right volume cm3':'Right.III',
    'Lobule III left volume cm3':'Left.III',
    'Lobule IV right volume cm3':'Right.IV',
    'Lobule IV left volume cm3':'Left.IV',
    'Lobule V right volume cm3':'Right.V',
    'Lobule V left volume cm3':'Left.V',
    'Lobule VI right volume cm3':'Right.VI',
    'Lobule VI left volume cm3':'Left.VI',
    'Crus I right volume cm3':'Right.Crus.I',
    'Crus I left volume cm3':'Left.Crus.I',
    'Crus II right volume cm3':'Right.Crus.II',
    'Crus II left volume cm3':'Left.Crus.II',
    'Lobule VIIB right volume cm3':'Right.VIIB',
    'Lobule VIIB left volume cm3':'Left.VIIB',
    'Lobule VIIIA right volume cm3':'Right.VIIIA',
    'Lobule VIIIA left volume cm3':'Left.VIIIA',
    'Lobule VIIIB right volume cm3':'Right.VIIIB',
    'Lobule VIIIB left volume cm3':'Left.VIIIB',
    'Lobule IX right volume cm3':'Right.IX',
    'Lobule IX left volume cm3':'Left.IX',
    'Lobule X right volume cm3':'Right.X',
    'Lobule X left volume cm3':'Left.X',
}

In [ ]:
# List to store DataFrames
all_data = []

# Loop through subjects and sessions
for sub in subjects_list:
    for ses in range(1,11):
        ses_str = f"{ses:02d}"
        
        # Get the matching file (for given subject and given session)
        dataFile = glob.glob(os.path.join(deepCeres, 'output', f"{sub}_ses-{ses_str}_T1w_job*_archive", "report_job*.csv"))[0]

        # Read CSV
        data = pd.read_csv(dataFile, sep=';')

        # Keep only the columns in col and rename them
        data = data[list(col.keys())].rename(columns=col)

        # Group I.II, III, IV and V into I.V
        data.insert(4, "Left.I.V", data["Left.I.II"] + data["Left.III"] + data["Left.IV"] + data["Left.V"])
        data.insert(4, "Right.I.V", data["Right.I.II"] + data["Right.III"] + data["Right.IV"] + data["Right.V"])
        data = data.drop(["Left.I.II", "Left.III", "Left.IV", "Left.V", "Right.I.II", "Right.III", "Right.IV", "Right.V"], axis = 1)

        # Add Subject ID and Session columns at the beginning
        data.insert(0, 'ses', ses_str)
        data.insert(0, 'ID', sub)

        # Append to the list
        all_data.append(data)

# Concatenate all DataFrames into one
deepCeres_df = pd.concat(all_data, ignore_index=True)

# Merge with age and sex
deepCeres_df= deepCeres_df.merge(subjects, on="ID", how="left")

# Sort columns
deepCeres_df = deepCeres_df[finalColumns]

# Save it
#deepCeres_df.to_csv(deepCeres+'all_volumes_cm3_deepCeres_ALLGOODDATA.csv', index=False)

deepCeres_df

,ID,ses,age,sex,ICV,Left.I.V,Right.I.V,Left.VI,Right.VI,Left.Crus.I,...,Left.VIIB,Right.VIIB,Left.VIIIA,Right.VIIIA,Left.VIIIB,Right.VIIIB,Left.IX,Right.IX,Left.X,Right.X
0,sub-0025428,01,28,F,1345.959321,4.873191,5.414353,8.033738,7.912000,11.220112,...,3.624231,4.161559,4.665031,4.945650,3.126599,3.269236,3.143665,3.075038,0.540614,0.610335
1,sub-0025428,02,28,F,1351.732163,4.844692,5.276966,7.886563,7.846398,10.797028,...,3.567770,4.070928,4.578120,5.000307,2.880662,3.242694,3.261492,3.021514,0.666475,0.618700
2,sub-0025428,03,28,F,1352.175522,4.859856,5.412166,8.075391,7.944715,11.112092,...,3.613228,4.015672,4.569171,4.982398,2.862894,3.143070,3.116752,2.888572,0.549295,0.541253
3,sub-0025428,04,28,F,1356.510079,4.818477,5.325366,8.020314,7.860640,11.267158,...,3.643464,4.126064,4.510401,5.122244,2.848390,3.217787,3.169573,2.979283,0.570411,0.569861
4,sub-0025428,05,28,F,1353.522653,4.743133,5.256121,7.954060,7.767444,11.154665,...,3.610375,4.000871,4.630597,4.910384,2.731005,3.175029,3.135751,3.005220,0.544502,0.555007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,sub-0025456,06,22,F,1503.705839,4.860340,5.504536,6.319404,6.711797,8.548162,...,3.429637,3.811698,4.721777,4.259799,3.287731,3.249950,2.926535,2.764473,0.585348,0.530854
176,sub-0025456,07,22,F,1509.097831,4.693826,5.355736,6.205669,6.562973,8.396047,...,3.318315,3.627354,4.508523,4.139027,3.332724,3.159412,2.880401,2.843219,0.595809,0.512680
177,sub-0025456,08,22,F,1503.597037,4.820297,5.429740,6.516051,6.578457,8.649354,...,3.530838,3.803651,4.805376,4.388898,3.342914,3.280508,2.963638,2.893670,0.599966,0.532318
178,sub-0025456,09,22,F,1513.758190,4.722178,5.487956,6.359184,6.616332,8.413233,...,3.367823,3.790635,4.714285,4.140280,3.253468,3.384724,2.882571,2.861117,0.580684,0.532209


In [ ]:
# Scale data
scaler = MinMaxScaler()

# Divide by ICV and scale
deepCeres_scaled = deepCeres_df.copy()
deepCeres_scaled.iloc[:, 4:] = scaler.fit_transform(deepCeres_scaled.iloc[:, 4:].div(deepCeres_scaled.ICV, axis=0))

# Save the dataframe
#deepCeres_scaled.to_csv(deepCeres+'all_volumes_cm3_deepCeres_scaled_ALLGOODDATA.csv', index=False)

deepCeres_scaled

,ID,ses,age,sex,ICV,Left.I.V,Right.I.V,Left.VI,Right.VI,Left.Crus.I,...,Left.VIIB,Right.VIIB,Left.VIIIA,Right.VIIIA,Left.VIIIB,Right.VIIIB,Left.IX,Right.IX,Left.X,Right.X
0,sub-0025428,01,28,F,0.0,0.817218,0.864766,0.963140,0.975421,0.964503,...,0.415756,0.812897,0.633153,0.838909,0.897829,0.831929,0.935120,0.995620,0.558674,0.979494
1,sub-0025428,02,28,F,0.0,0.785214,0.789873,0.907428,0.943468,0.881528,...,0.370906,0.748719,0.592104,0.851420,0.740170,0.804631,1.000000,0.955676,1.000000,1.000000
2,sub-0025428,03,28,F,0.0,0.794006,0.852090,0.964535,0.974193,0.936366,...,0.398483,0.715251,0.588092,0.844110,0.728798,0.736893,0.909367,0.875513,0.580758,0.723013
3,sub-0025428,04,28,F,0.0,0.757235,0.803696,0.939789,0.939152,0.957325,...,0.410060,0.772740,0.560005,0.890284,0.714453,0.780242,0.935905,0.924109,0.649658,0.818567
4,sub-0025428,05,28,F,0.0,0.715354,0.776911,0.924904,0.914823,0.941906,...,0.394468,0.704142,0.609899,0.815352,0.646996,0.756267,0.919237,0.943541,0.561707,0.770104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,sub-0025456,06,22,F,0.0,0.477115,0.636574,0.230824,0.361481,0.333394,...,0.068969,0.367453,0.464041,0.413374,0.785636,0.588469,0.607823,0.634495,0.498861,0.495095
176,sub-0025456,07,22,F,0.0,0.370374,0.566178,0.193349,0.311767,0.304567,...,0.000000,0.262524,0.384881,0.367790,0.803716,0.526868,0.576284,0.671410,0.525620,0.430923
177,sub-0025456,08,22,F,0.0,0.453998,0.605385,0.285176,0.323140,0.349512,...,0.125777,0.363320,0.493013,0.456892,0.815924,0.607099,0.628681,0.704144,0.545944,0.499912
178,sub-0025456,09,22,F,0.0,0.378390,0.614347,0.230148,0.321253,0.303192,...,0.021837,0.342865,0.450650,0.363939,0.755106,0.656404,0.572550,0.676285,0.471501,0.488104
